# Pre-processing and synchronizing ephys data: an example notebook

The objective of this notebook is to be able to take all the output files that are generated by the ephys setup and Ronan cottage and synchronizing all of them to the same clock.

We are using session S20230323 from BRAC4779.2a [HELMET], the first session we recorded. 

## Running list of things ot improve on

- As a small note, we have to re-write the bonsai workflow to record the cameras in the arena in a convincing way, so that it inherits the name from the stimulus workflow ideally, like in Yiran's case. Work on that. 
- The outputs should all go to the same recording folder if they're part of the same recording. Like in Yiran's case. 
- We want to save the raw output from the chip, so we will have to find a way to remap it later, I guess. DONE
- We want to use a bonsai workflow for the stimuli that is able to save a NewParams matrix. DONE
- HF cameras need to be timestamped. DONE
- Do not name the animal 'cowboy' ever again
- The camera is called letfeye_camera instead of lefteye_camera

The exposures of all the cameras are 7000 micros. That's not the reason for the frame drop in face cam. 
I'm trying for 20230406 to only visualize one cam, behaviour_cam


In [ ]:
from pathlib import Path
import cottage_analysis.io_module.onix as onix
import cottage_analysis.io_module.harp as harp
import cottage_analysis.ephys.preprocessing as prp
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd 
from scipy import stats


## Which files were outputted this time?

- Three different folders and some loose files 
    - Outputs of the AcquireEphys
    - Outputs of stimulus
    - Outputs of HF cameras
    - Outputs of FM cameras

In [ ]:
processed_path = Path('/camp/lab/znamenskiyp/home/shared/projects/blota_onix_pilote/')
data_path = Path('/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/')
mouse = 'BRAC7448.2d'
session = 'S20230404'
session_path = data_path / mouse / session

processed_dir = processed_path/mouse/session
processed_dir.mkdir(exist_ok=True)

ephys = 'R110119'
stimulus = 'R110246_SpheresPermTubeReward'
headfixed_cam = '110357' 

ephys_path = session_path / ephys
stimulus_path = session_path / stimulus
headfixed_cam_path = session_path / headfixed_cam

## Importing the ephys outputs (rhd2164)

Ephys outputs start with rhd2164, which is the intan digital electrophysiology interface chip on the headstage. There are three files created, one is aux, one is clock, one is ephys and one is first_time. 

Antonin wrote a function which takes them all together and makes a memmap object with them, so that they can be easily accessed. 

In [ ]:
processed_ephys = onix.load_rhd2164(ephys_path)
processed_ephys

Let's attempt to read the ephys outputs!

import matplotlib.pyplot as plt 
import numpy as np
import os
import spikeinterface.extractors as se
import pickle
import pandas as pd
import spikeinterface as si

num_channels = 64
sampling_frequency = 30000
gain_to_uV = 0.195
offset_to_uV = -2**15*gain_to_uV
dtype="uint16"#This could be sensitive: our chip writes in uint16, had to change to int16 for kilosort (?)
time_axis = 0

path_to_openephys = '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7449.2a/S20230323/R115806/rhd2164-ephys_2023-03-23T11_58_06.raw'

recording = si.read_binary(path_to_openephys, num_chan=num_channels, sampling_frequency=sampling_frequency,
                            dtype=dtype, gain_to_uV=gain_to_uV, offset_to_uV=offset_to_uV,
                            time_axis=time_axis)



import spikeinterface.widgets as sw

#trace_snippet = recording.get_traces(start_frame=int(fs*0), end_frame=int(fs*2))
#print('Traces shape:', trace_snippet.shape)

import scipy.signal
from spikeinterface.preprocessing import (bandpass_filter, notch_filter, common_reference,
                                          remove_artifacts, preprocesser_dict)
recording_bp = bandpass_filter(recording, freq_min=200, freq_max=10000)

recording_cmr = common_reference(recording_bp, reference='global', operator='median')


channel_ids = range(30, 40)
channel_ids

w_ts = sw.plot_timeseries(recording_cmr, channel_ids=channel_ids, time_range=(2000.8, 2001), return_scaled=True)

## Loading lighthouse data (ts4231)

Three different lighthouse-generated files, one per diode. ts4231 is the name of the photodiode

In [ ]:
processed_photodiode = onix.load_ts4231(ephys_path)
processed_photodiode

In [ ]:
fig, axs = plt.subplots(3, 1, figsize = (9, 7))

fig.suptitle('Value of x for the three photodiodes in the headstage')
axs[0].plot(processed_photodiode[1]['clock'], processed_photodiode[1]['x'])
axs[1].plot(processed_photodiode[2]['clock'], processed_photodiode[2]['x'])
axs[2].plot(processed_photodiode[3]['clock'], processed_photodiode[3]['x'])


## Loading IMU data (bno055)

We have nothing to load the IMU data. Should I write it? Wrote it on onix. 

In [ ]:
processed_bno = onix.load_bno055(ephys_path)
processed_bno.keys()

quaternion = processed_bno['quaternion']
quaternion = quaternion/2**14
linear = processed_bno['linear']
linear = linear / 100
euler = processed_bno['euler']
euler = euler / 16
gravity = processed_bno['gravity']
gravity = gravity / 100

Let's see how they look. 

In [ ]:
minutes = len(linear[:,1])/100/60

print(f'The recording was {minutes} min in length')

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 9))

fig.suptitle('BNO055 (timestamps 20000-201000)')

timeslice=range(170000, 170100)

axs[0, 0].plot(quaternion[timeslice, 3])
axs[0, 0].plot(quaternion[timeslice, 0])
axs[0, 0].plot(quaternion[timeslice, 1])
axs[0, 0].plot(quaternion[timeslice, 2])
axs[0, 0].set_title('Quaternion')

axs[0, 1].plot(linear[timeslice, 0])
axs[0, 1].plot(linear[timeslice, 1])
axs[0, 1].plot(linear[timeslice, 2])
axs[0, 1].set_title('Linear acceleration')

axs[1, 1].plot(euler[timeslice, 0])
axs[1, 1].plot(euler[timeslice, 1])
axs[1, 1].plot(euler[timeslice, 2])
axs[1, 1].set_title('Euler Vector')

axs[1, 0].plot(gravity[timeslice, 0])
axs[1, 0].plot(gravity[timeslice, 1])
axs[1, 0].plot(gravity[timeslice, 2])
axs[1, 0].set_title('Gravity vector')

And extract the heading from this

In [ ]:
from math import atan2

fq0=quaternion[:,0]
fq1=quaternion[:,1]
fq2=quaternion[:,2]
fq3=quaternion[:,3]
heading = np.zeros(len(fq0))
#heading[sample] = [atan2((2*(quaternion[0]*quaternion[3]+quaternion[1]*quaternion[2])), (1-2*(quaternion[2]**2+quaternion[3]**2))) for sample in quaternion[0]]

for i in range(len(fq0)):
    q0, q1, q2, q3 = fq0[i], fq1[i], fq2[i], fq3[i]
    heading[i]=atan2((2*(q0*q3+q1*q2)), (1-2*(q2**2+q3**2)))



In [ ]:
plt.plot(heading[timeslice])


In [ ]:
processed_bno['real_time'] = (processed_bno['onix_time']-processed_bno['onix_time'][0])/250e6

## Loading the stimulus logs and synchronizing them to the ephys clock

There is no NewParams matrix, but there are other stuff which I can try to sync, like the rotary encoder

In [ ]:
param_file =  stimulus_path / 'NewParams.csv'
processed_stimulus = pd.read_csv(param_file)
processed_stimulus

## Loading the HARP

Impossible to do it de novo in jupyter. See the scrap in the notebook folder for this. 

In [ ]:
processed_messages = processed_path / mouse / session / stimulus / 'processed_harp.npz'

if Path(processed_messages).is_file():
    harp_message = dict(np.load(processed_messages));
else:
    print('Impossible to read a processed HARP file. If this is  anew recording and you are on a notebook use the scrap on the notebooks folder')

In [ ]:

harp_message.keys()

Synchronizing the HARP

In [ ]:
harp_message, harp2onix = prp.sync_harp2onix(harp_message)
processed_stimulus['onix_time'] = harp2onix(processed_stimulus['HarpTime'])
#Attention: this function takes a harp time and outputs a time NOT IN ONIX TIME but in s since the harp started counting
# make a speed out of rotary increament
mvt = np.diff(harp_message['rotary']) #The rotary shows position of the machine. One has to play with it to turn it into something that can be actual speed.
rollover = np.abs(mvt > 40000) #Create a mask to find the points in which the wheel has made a turn, thus the difference is huge
mvt[rollover] -= 2**16 * np.sign(mvt[rollover]) #substracts the value of the rollover from the difference at the points where there's a rollover.
# The rotary count decreases when the mouse goes forward
mvt *= -1 #inverts the direction of the mouse.
harp_message['mouse_speed'] = np.hstack([0, mvt])  # 0-padding to keep constant length

In [ ]:
processed_stimulus

## Loading the breakout board 

Analog and digital inputs to the board

In [ ]:
processed_breakout = onix.load_breakout(ephys_path)
print(processed_breakout.keys())
processed_breakout['aio']


In [ ]:
processed_breakout['dio']

In [ ]:
processed_breakout['dio']
print(set(processed_breakout['dio']['Buttons']))
start_headfixed = np.where(processed_breakout['dio']['Buttons'] == 8)
start_headfixed = processed_breakout['dio']['Clock'][start_headfixed[0][0]]
print(f'The head-fixed recording started at clock time {start_headfixed}')
end_headfixed = np.where(processed_breakout['dio']['Buttons'] == 16)
end_headfixed = processed_breakout['dio']['Clock'][end_headfixed[0][0]]
print(f'The head-fixed recording ended at clock time {end_headfixed}')
start_freely_moving = np.where(processed_breakout['dio']['Buttons'] == 32)
start_freely_moving = processed_breakout['dio']['Clock'][start_freely_moving[0][0]]
print(f'The freely moving recording started at clock time {start_freely_moving}')


In [ ]:
processed_breakout['dio']['real_time'] = (processed_breakout['dio']['Clock']-processed_breakout['dio']['Clock'][0])/250e6

## Loading the freely moving cameras and synchronizing them to the ephys clock

Let's have a look at the clock they have. Read the timestamp dataframes as pandas. Have a look to see if the vseod have the same number of frames as the cameras. 

In [ ]:
camera_dir = session_path / 'R123342_cowboy'
acquisition='freely_moving'

def load_camera_times(camera_dir, acquisition):
    if acquisition == 'freely_moving':
        camlist = ['cam1_camera', 'cam2_camera', 'cam3_camera']
    if acquisition == 'headfixed':
        camlist = ['letfeye_camera', 'righteye_camera', 'face_camera', 'behaviour_camera']
    folder = Path(camera_dir)
    if not folder.is_dir():
        raise IOError('%s is not a directory' % folder)
    output = dict()
    for cam in camlist:
        cam_folder = folder/cam
        valid_files = list(cam_folder.glob('*timestamp*'))
        if not len(valid_files):
            raise IOError(f'Could not find any timestamp files in {folder}')
        for possible_file in valid_files:
            possible_file = str(possible_file)
            if cam in possible_file:
                output[cam] = pd.read_csv(possible_file)
    return(output)

processed_fm = load_camera_times(camera_dir, acquisition)
processed_fm

## Camera synchronization

Have to find when FM cameras come online with the DI0 in the digital inputs of the breakout board. 

In [ ]:
fm_triggers = np.where(processed_breakout['dio']['DI0']==1)
t0 = processed_breakout['dio']['Clock'][fm_triggers[0][0]]
fm_trigger_clock = list(processed_breakout['dio']['Clock'][processed_breakout['dio']['DI0']==1])
fm_camera_clock = list(processed_fm['cam1_camera']['camera_timestamp'])
#I'm doing : trigger_clock=constant+camera_clock*slope
slope = (fm_trigger_clock[-1]-fm_trigger_clock[0])/(fm_camera_clock[-1]-fm_camera_clock[0])
def cam2onix(data, slope = slope, t0 = t0):
    return t0+(slope*data)
camlist = ['cam1_camera', 'cam2_camera', 'cam3_camera']
for cam in camlist:
    processed_fm[cam]['onix_time'] = cam2onix(processed_fm[cam]['camera_timestamp'])


In [ ]:
print(len(fm_trigger_clock))
print(len(fm_camera_clock))
print(slope)
print(t0)
plt.plot(processed_breakout['dio']['Clock'], processed_breakout['dio']['DI2'])
plt.plot(processed_breakout['dio']['Clock'], processed_breakout['dio']['DI0'])
#plt.plot(list(processed_fm['cam1_camera']['onix_time']), np.ones_like(list(processed_fm['cam1_camera']['onix_time'])))
plt.axvline(start_freely_moving)

In [ ]:
processed_breakout['dio']['Clock'][fm_triggers[0][-1]]
fm_trigger_clock


Let's have a look at the camera triggers

In [ ]:
dio = processed_breakout['dio']

In [ ]:
trigger = np.array(dio['DI0'])
is_noise = np.diff(dio['Clock'])<3000
print(is_noise)
#plt.hist(np.diff(dio['Clock'][1:][is_noise]), bins = np.linspace(0,200,100))

trigger[1:][is_noise] = trigger[:-1][is_noise]
#plt.hist(np.diff(is_noise)) 

on_trigger = dio['Clock'][1:][np.diff(trigger)==1]
on_dio = dio['Clock'][1:][np.diff(dio['DI0'])==1]

print(len(on_trigger))
print(len(on_dio))

bins = np.arange(15)


plt.hist(np.diff(on_dio)/250e3, bins=bins, histtype = 'step', label = 'dio', lw = 2)
plt.hist(np.diff(on_trigger)/250e3, bins = bins, histtype = 'step', label = 'trigger')
plt.legend()


I want to find a way to filter the DI0 output, which is made of a bunch of 7 microseconds sequence of 1s, so that I can get rid of high-frequency noise

The onix clock fires at 256e6 Hz, while the digital inputs fire at 5e6 Hz. 

How does the distribution of delays between turning-on events (from 0 to 1) look like?

In [ ]:
fm_triggers = dio['DI0']

on_triggers = dio['Clock'][1:][np.diff(fm_triggers)==1]
off_triggers = dio['Clock'][1:][np.diff(fm_triggers)==255]

print(f'The number of turn-on events in dio is {len(on_triggers)}')
print(f'The number of turn-off events in dio is {len(off_triggers)}')

import cv2

camera_dir ='/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7448.2d/S20230404/R123342_cowboy/cam3_camera/cam3_camera_2023-04-04T12_33_42.mp4'

cap = cv2.VideoCapture(camera_dir)

n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print(f'There are {n_frames} frames in the acquired video, a difference of {n_frames-len(on_triggers)} with the turn-on events')

cam1 = processed_fm['cam1_camera']

n_metadata_frames = len(cam1['camera_timestamp'])

print(f'The number of frames in the camera metadata is {n_metadata_frames}')

frame_ids = cam1['frame_id'].values[-1]-cam1['frame_id'].values[0]

print(f'However, the difference between the first and last frame_id in the camera is {frame_ids}')
print(f'This is a difference of {n_frames-frame_ids} with the frames in the video and {len(on_triggers)-frame_ids} with the turn-on events')

### Hunting for noise in the DIO signal to discard fake turn-on events

In [ ]:
plt.hist((off_triggers.values-on_triggers.values)/250e3)
plt.xlabel('Length (ms)')
plt.ylabel('Frequency')
plt.title('How long is a DI0 pulse?')
longpulses = np.sum(((off_triggers.values-on_triggers.values)/250e3)>3)
print(f'There are {longpulses} pulses over 3ms long. This is {(100*longpulses)/len(on_triggers)}% of turn-on events')

There is a group of very short pulses, and a large group of roughly 6ms pulses, which is our exposition value. 

In [ ]:
plt.hist(np.diff(on_triggers), bins = np.linspace(0,500,100))
plt.xlabel('Length (onix oscillations)')
plt.ylabel('Frequency')
plt.title('How long is a short (fake) DI0 pulse? Very short')

print(np.sum((np.diff(on_triggers)/250e6)>0.002))

Then. What is a reasonable, however costly, algorithm to get rid of this noise? Two steps: 

- For every value, if most of the values in a +-400 onix oscillations window from it are different from it, it switches to the other value. 
- This step gets rid of values that are in the middle of either a pulse or a silent period, not the ones at the edges between a pulse or a silent period. But for those values, it's enough to filter pulses so that we only keep the ones longer than 3ms, because they will for sure be tiny. 

Alternatively, Antonin suggests median filter with a 400 window. Let's try

import scipy.signal

DI_list = ['DI0', 'DI1', 'DI2', 'DI3', 'DI4', 'DI5', 'DI6', 'DI7']

filtered_dio = dio

for DI in DI_list:
    filtered_DI = scipy.signal.medfilt(dio[DI], 399)
    filtered_dio[DI] = filtered_DI


In [ ]:
window = 399
dio_clock = dio['Clock']
start_batch = dio_clock.searchsorted(dio_clock-window)
#pandas searchsorted method = Find the indices into a sorted Series self such that, 
#if the corresponding elements in value were inserted before the indices, the order of self would be preserved.
#end_batch = np.clip(dio_clock.searchsorted(dio_clock+window), 0, len(dio_clock))
end_batch = dio_clock.searchsorted(dio_clock+window)
batch_size = end_batch - start_batch
batch_ones = [np.sum(dio['DI0'][s:e]) for s,e in zip(start_batch, end_batch)]
median_filtered_1 = batch_ones > batch_size/2
median_filtered_0 = batch_ones < batch_size/2



In [ ]:
median_filtered_0 = batch_ones < batch_size/2
filtered_DI0_1 = np.where(median_filtered_1==True, 1, dio['DI0'])
filtered_DI0 = np.where(median_filtered_0==True, 0, filtered_DI0_1)

In [ ]:
filtered_dio = dio
filtered_dio['DI0'] = filtered_DI0

fm_triggers = filtered_dio['DI0']

on_triggers = filtered_dio['Clock'][1:][np.diff(fm_triggers)==1]
off_triggers = filtered_dio['Clock'][1:][np.diff(fm_triggers)==255]


print(f'The number of turn-on events in dio is {len(on_triggers)}')
print(f'The number of turn-off events in dio is {len(off_triggers)}')

plt.hist((off_triggers.values-on_triggers.values)/250e3)
plt.xlabel('Length (ms)')
plt.ylabel('Frequency')
plt.title('How long is a DI0 pulse?')
longpulses = np.sum(((off_triggers.values-on_triggers.values)/250e3)>3)
print(f'There are {longpulses} pulses over 3ms long. This is {(100*longpulses)/len(on_triggers)}% of turn-on events')
print(f'There are {n_frames} frames in the acquired video, a difference of {n_frames-len(on_triggers)} with the turn-on events')

cam1 = processed_fm['cam2_camera']

frame_ids = cam1['frame_id'].values[-1]-cam1['frame_id'].values[0]

print(f'However, the difference between the first and last frame_id in the camera is {frame_ids}')
print(f'This is a difference of {n_frames-frame_ids} with the frames in the video and {len(on_triggers)-frame_ids} with the turn-on events')

Lag between two camera timestamps: 

About the clock. If 1.022*10^7 are the oscillations between two frames, and one frame is 1/100s, then the clock has 1002243200.0 cycles/second

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 13))

fig.suptitle('Time between two frames (s)')

camlist = ['cam1_camera', 'cam2_camera', 'cam3_camera']

for i, cam in enumerate(camlist):
    lag = np.diff(processed_fm[cam]['camera_timestamp'])
    lag=lag/1002243200
    axs[i].hist(lag)
    axs[i].set_title(cam)




Difference between two frames (should be always one, otherwise there is frame drop)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 13))

camlist = ['cam1_camera', 'cam2_camera', 'cam3_camera']
fig.suptitle('Space between two frame_ids')

for i, cam in enumerate(camlist):
    lag = np.diff(processed_fm[cam]['frame_id'])
    axs[i].hist(lag)
    axs[i].set_title(cam)


Ratio of camera timestamps between two frames to number of triggers between two frames. Should be the frame rate (timestamps/frame), except if frame drop

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 13))

fig.suptitle('Frame rate (seconds/triggered frame)')

camlist = ['cam1_camera', 'cam2_camera', 'cam3_camera']

for i, cam in enumerate(camlist):
    lag = np.diff(processed_fm[cam]['camera_timestamp'])/1002243200
    lag_frame = np.diff(processed_fm[cam]['frame_id'])
    lag=lag/lag_frame
    print(stats.mode(lag))
    axs[i].hist(lag)
    axs[i].set_title(cam)

In [ ]:
import cv2

camera_dir ='/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7448.2d/S20230404/R123342_cowboy/cam1_camera/cam1_camera_2023-04-04T12_33_42.mp4'

cap = cv2.VideoCapture(camera_dir)

n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
n_frames

print(len(processed_fm['cam1_camera']['camera_timestamp']))
print(n_frames)

fm_triggers = processed_breakout['dio']['DI0']

fm_triggers.value_counts()

on_triggers = processed_breakout['dio']['Clock'][1:][np.diff(fm_triggers)==1]
off_triggers = processed_breakout['dio']['Clock'][1:][np.diff(fm_triggers)==255]


print(len(on_triggers))
print(len(off_triggers))


cam1 = processed_fm['cam1_camera']

cam1['frame_id'].values[-1]-cam1['frame_id'].values[0]

In [ ]:
plt.hist((off_triggers.values-on_triggers.values)/250e3)
np.sum(((off_triggers.values-on_triggers.values)/250e3)>3)

In [ ]:
plt.hist(np.diff(on_triggers), bins = np.linspace(0,500,100))

print(np.sum((np.diff(on_triggers)/250e6)>0.002))

## Loading the head-fixed cameras and synchronizing them to the ephys clock

In [ ]:
camera_dir = session_path / 'R110708_cowboy'
acquisition='headfixed'

def load_camera_times(camera_dir, acquisition):
    if acquisition == 'freely_moving':
        camlist = ['cam1_camera', 'cam2_camera', 'cam3_camera']
    if acquisition == 'headfixed':
        camlist = ['letfeye_camera', 'righteye_camera', 'face_camera', 'behaviour_camera']
    folder = Path(camera_dir)
    if not folder.is_dir():
        raise IOError('%s is not a directory' % folder)
    output = dict()
    for cam in camlist:
        cam_folder = folder/cam
        valid_files = list(cam_folder.glob('*timestamp*'))
        if not len(valid_files):
            raise IOError(f'Could not find any timestamp files in {folder}')
        for possible_file in valid_files:
            possible_file = str(possible_file)
            if cam in possible_file:
                output[cam] = pd.read_csv(possible_file)
    return(output)

processed_hf = load_camera_times(camera_dir, acquisition)
processed_hf
#ignore index or keep it

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(10, 18))

fig.suptitle('Time between two frames (s)')

camlist = ['letfeye_camera', 'righteye_camera', 'face_camera', 'behaviour_camera']

for i, cam in enumerate(camlist):
    lag = np.diff(processed_hf[cam]['camera_timestamp'])
    lag=lag/1002243200
    axs[i].hist(lag)
    axs[i].set_title(cam)

#face exposure?? run only this one to see if it's still losing it

## Diagnostics

In [ ]:
# plot what I did
harp_onix_clock = harp_message['onix_clock']
real_time = np.arange(len(harp_onix_clock)) * 10e-3  # assume a perfect 100Hz

ax = fig.add_subplot(2, 2, 2)
ax.clear()
ax.hist(np.diff(harp_onix_clock)*1000)
ax.set_xlabel('Intertrigger time (ms)')

ax = fig.add_subplot(2, 2, 3)
ax.clear()
ax.plot(harp_onix_clock, real_time, '.')
ax.set_xlabel('Harp time')
ax.set_ylabel('Onix time')

ax = fig.add_subplot(2, 2, 4)
ax.clear()
ax.plot(real_time, (harp2onix(harp_onix_clock) - real_time) * 1000, 'o')
ax.set_xlabel('Onix time')
ax.set_ylabel('Error (ms)')

In [ ]:
depth_trace = processed_stimulus.Depth.values
corridor_starts, = np.where(np.hstack([True, np.diff(depth_trace) > 4000]))
corridor_ends, = np.where(np.hstack([False, np.diff(depth_trace) < -4000]))
# make a simpler dataframe
corridor_df = []
for s, e in zip(corridor_starts, corridor_ends):
    corridor_df.append(dict(start_index=s, start_time=processed_stimulus.onix_time.iloc[s],
                            end_index=e, end_time=processed_stimulus.onix_time.iloc[e],
                            depth=processed_stimulus.Depth[s]))
corridor_df = pd.DataFrame(corridor_df)

fig = plt.figure()
fig.clf()
fig.set_size_inches(18.5, 18.5)
ax0 = fig.add_subplot(5, 1, 1)
ax0.plot(processed_stimulus.onix_time.values, processed_stimulus.Depth.values)
depths = corridor_df.depth.unique()
cmap = plt.get_cmap('Set2', len(depths))
depth_color = {d: cmap(i) for i, d in enumerate(depths)}
ax0.set_ylabel('Depth (cm)')
ax0.set_ylim(0, 630)
ax1 = fig.add_subplot(5, 1, 2, sharex=ax0)
#ax1.plot(harp_message['analog_onix_time'], harp_message['mouse_speed'], alpha=0.2)
# make a moving average on a 100ms window (assuming 1kHz sampling)
w = int(0.5 * 1000)
cs = np.cumsum(harp_message['mouse_speed'])
smooth_speed = (cs[w:] - cs[:-w]) / w
ax1.plot(harp_message['analog_time_onix'][int(w/2):-int(w/2)], smooth_speed)
ax1.set_ylabel('Running speed')
ax1.set_ylim([-2, 3])
ax0.set_xlim([0, 6800])
for x in [ax0, ax1]:
    for i_c, cdf in corridor_df.iterrows():
        x.axvspan(cdf.start_time, cdf.end_time, alpha=0.2, color=depth_color[cdf.depth])
ax2 = fig.add_subplot(5, 1, 3, sharex=ax0)
ax2.plot(processed_bno['real_time'], linear[0:len(processed_bno['real_time']),1])
ax2.plot(processed_bno['real_time'], linear[0:len(processed_bno['real_time']),2])
ax2.plot(processed_bno['real_time'], linear[0:len(processed_bno['real_time']),0])
ax2.set_ylabel('Linear acceleration')
ax3 = fig.add_subplot(5, 1, 4, sharex=ax0)
ax3.plot(processed_bno['real_time'], gravity[0:len(processed_bno['real_time']),1])
ax3.plot(processed_bno['real_time'], gravity[0:len(processed_bno['real_time']),2])
ax3.plot(processed_bno['real_time'], gravity[0:len(processed_bno['real_time']),0])
ax3.set_ylabel('Gravity vector')
ax4 = fig.add_subplot(5, 1, 5, sharex = ax0)
ax4.plot(processed_breakout['dio']['real_time'], processed_breakout['dio']['DI2'])
ax4.plot(processed_breakout['dio']['real_time'], processed_breakout['dio']['DI0'])
ax4.set_ylabel('Camera triggers')

In [ ]:
processed_breakout['dio']['Clock']

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 9))

fig.suptitle('BNO055 (timestamps 20000-201000)')

timeslice=range(170000, 170100)

axs[0, 0].plot(quaternion[timeslice, 3])
axs[0, 0].plot(quaternion[timeslice, 0])
axs[0, 0].plot(quaternion[timeslice, 1])
axs[0, 0].plot(quaternion[timeslice, 2])
axs[0, 0].set_title('Quaternion')

axs[0, 1].plot(linear[timeslice, 0])
axs[0, 1].plot(linear[timeslice, 1])
axs[0, 1].plot(linear[timeslice, 2])
axs[0, 1].set_title('Linear acceleration')

axs[1, 1].plot(euler[timeslice, 0])
axs[1, 1].plot(euler[timeslice, 1])
axs[1, 1].plot(euler[timeslice, 2])
axs[1, 1].set_title('Euler Vector')

axs[1, 0].plot(gravity[timeslice, 0])
axs[1, 0].plot(gravity[timeslice, 1])
axs[1, 0].plot(gravity[timeslice, 2])
axs[1, 0].set_title('Gravity vector')